In [1]:
# !pip install websockets

In [1]:
import pymssql
from vosk import Model, KaldiRecognizer, SetLogLevel
import wave
import json
import asyncio
import websockets

In [2]:
# cpu approach
model_path = '/mnt/share/audio_call/model_v0/model'
model = Model(model_path)

In [3]:
def connect_sql():

    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'
    
    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()
    
    return pymssql.connect(
        server=sql_server,
        user=sql_login,
        password=sql_pass,
        database=sql_name,
    )

In [4]:
def get_file(source_id):
    
    conn = connect_sql()
    cursor = conn.cursor()
    
    # middle date
    sql_query = "select"
    sql_query += " CAST(AVG(CAST(record_date-0.9 AS FLOAT)) AS DATETIME),"
    sql_query += " CAST(AVG(CAST(record_date+0.1 AS FLOAT)) AS DATETIME)"
    sql_query += " from queue"
    sql_query += " where source_id = '"+source_id+"' and duration>600;"
    cursor.execute(sql_query)
    for row in cursor.fetchall():
        min_time = str(row[0])[:19]
        max_time = str(row[1])[:19]
        print(min_time)
    
    # file name and path from queue
    sql_query = "select top 1 filepath, filename from queue "
    sql_query += "where source_id = '"+source_id+"' "
    sql_query += " and record_date > '" + min_time + "' "
    sql_query += " and record_date < '" + max_time + "' "
    sql_query += " order by record_date, filename;"
    cursor.execute(sql_query)
    for row in cursor.fetchall():
        original_file_path = row[0]
        original_file_name = row[1]
        
    return original_file_path, original_file_name

In [6]:
#original_file_path, original_file_name = get_file('1')
#original_file_path, original_file_name

### MRM records

In [5]:
original_file_path, original_file_name = get_file('2')

2021-08-19 12:56:06


In [6]:
original_file_path, original_file_name

('/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/',
 'a2021-08-19t12:56:08b_c9163161554d_e4959322127f_g1629366966.13780h-in.wav')

In [57]:
def cpu_approach(model, original_file_path, original_file_name):
    # read file
    wf = wave.open(original_file_path + original_file_name, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    phrases_count = 0
    confidences = []
    phrases = []
    while True:

        conf_score = []

        data = wf.readframes(4000)
        if len(data) == 0:
            break

        if rec.AcceptWaveform(data):
            accept = json.loads(rec.Result())
            print('*', accept)
            if accept['text'] != '':

                accept_start = str(accept['result'][0]['start'])
                accept_end = accept['result'][-1:][0]['end']
                accept_text = str(accept['text'])

                for result_rec in accept['result']:
                    conf_score.append(float(result_rec['conf']))
                conf_mid = str(sum(conf_score)/len(conf_score))
                confidences.append(sum(conf_score)/len(conf_score))
                print(accept_start, accept_text)
                phrases.append({
                    'time': accept_start,
                    'conf': conf_mid,
                    'text': accept_text
                })
                phrases_count += 1
    print('end')
    return phrases

In [32]:
phrases_cpu = cpu_approach(model, original_file_path, original_file_name)
phrases_cpu

* {'text': ''}
end


[]

In [8]:
#async def gpu_approach(original_file_path, original_file_name):
server_ip = '10.2.5.212'
uri = 'ws://' + server_ip + ':2700'
phrases_count = 0
confidences = []
phrases = []
async with websockets.connect(uri) as websocket:
    wf = open(original_file_path + original_file_name, "rb")
    while True:
        conf_score = []
        data = wf.read(8000)
        if len(data) == 0:
            break
        await websocket.send(data)
        accept = json.loads(await websocket.recv())
        print('*', accept)
        if len(accept)>1 and accept['text'] != '':

            accept_start = str(accept['result'][0]['start'])
            accept_end = accept['result'][-1:][0]['end']
            accept_text = str(accept['text'])

            for result_rec in accept['result']:
                conf_score.append(float(result_rec['conf']))
            conf_mid = str(sum(conf_score)/len(conf_score))
            confidences.append(sum(conf_score)/len(conf_score))
            print(accept_start, accept_text)
            phrases.append({
                'time': accept_start,
                'conf': conf_mid,
                'text': accept_text
            })
            #phrases_count += 1
        """if len(accept)>1 and accept['text'] != '':
            pass
            #file_data.append(accept['text'])
            #textfile.write(str(accept['text'])+'\n')
        else:
            [[key, value]] = accept.items()
            if key == 'text' and len(value):
                print('===', value)"""
        
    await websocket.send('{"eof" : 1}')
    print(await websocket.recv())
            
#        return accept
#asyncio.get_event_loop().run_until_complete(
#    gpu_approach(original_file_path, original_file_name)
#)

* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': 'думаю'}
* {'partial': 'доброе'}
* {'partial': 'добрый мастер'}
* {'partial': 'думаю есть мастера по холодильникам и не'}
* {'partial': 'думаю есть мастера по холодильникам меня зовут андрей'}
* {'partial': 'думаю есть мастера по холодильникам меня зовут андрей что'}
* {'partial': 'думаю есть мастера по холодильникам меня зовут андрей что'}
* {'result': [{'conf': 0.840414, 'end': 7.353184, 'start': 6.99, 'word': 'думаю'}, {'conf': 0.880365, 'end': 7.65, 'start': 7.353184, 'word': 'есть'}, {'conf': 1.0, 'end': 8.1, 'start': 7.71, 'word': 'мастера'}, {'conf': 1.0, 'end': 8.19, 'start': 8.1, 'word': 'по'}, {'conf': 1.0, 'end': 8.82, 'start': 8.19, 'word': 'холодильникам'}, {'conf': 1.0, 'end': 9.06, 'start': 8.82

* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
{
  "text" : ""
}


In [9]:
phrases

[{'time': '6.99',
  'conf': '0.9114615',
  'text': 'думаю есть мастера по холодильникам меня зовут андрей что ложка'},
 {'time': '11.55', 'conf': '0.859714', 'text': 'что такое случилось'},
 {'time': '23.31', 'conf': '0.515987', 'text': 'о'},
 {'time': '30.51', 'conf': '0.888716', 'text': 'знаешь'},
 {'time': '34.17',
  'conf': '0.8377475999999999',
  'text': 'ну смотрите могу через где-нибудь в течение нечасто подъехать у вас улице раменский том что'},
 {'time': '44.07',
  'conf': '0.7280962',
  'text': 'подъезд и этаж квартирный код'},
 {'time': '50.46', 'conf': '0.963953', 'text': 'ну'},
 {'time': '54.78', 'conf': '1.0', 'text': 'такой'},
 {'time': '59.01', 'conf': '0.701221', 'text': 'ну'},
 {'time': '61.62', 'conf': '0.695498', 'text': 'вызов'},
 {'time': '66.81', 'conf': '1.0', 'text': 'словом вызов без вызова'},
 {'time': '72.81', 'conf': '1.0', 'text': 'квартира номер сто двадцать два'},
 {'time': '77.871562', 'conf': '0.662888', 'text': 'до встречи'},
 {'time': '79.44', 'conf'

In [11]:
### sync gpu
async def gpu_approach(temp_file_path, temp_file_name):
    #original_file_path = '/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/'
    #original_file_name = 'a2021-08-19t12:56:08b_c9163161554d_e4959322127f_g1629366966.13780h-in.wav'
    server_ip = '10.2.5.212'
    uri = 'ws://' + server_ip + ':2700'
    phrases_count = 0
    confidences = []
    phrases = []
    async with websockets.connect(uri) as websocket:
        wf = open(original_file_path + original_file_name, "rb")
        while True:
            conf_score = []
            data = wf.read(8000)
            if len(data) == 0:
                break
            await websocket.send(data)
            accept = json.loads(await websocket.recv())
            print('*', accept)
            if len(accept)>1 and accept['text'] != '':

                accept_start = str(accept['result'][0]['start'])
                accept_end = accept['result'][-1:][0]['end']
                accept_text = str(accept['text'])

                for result_rec in accept['result']:
                    conf_score.append(float(result_rec['conf']))
                conf_mid = str(sum(conf_score)/len(conf_score))
                confidences.append(sum(conf_score)/len(conf_score))
                print(accept_start, accept_text)
                phrases.append({
                    'time': accept_start,
                    'conf': conf_mid,
                    'text': accept_text
                })

        await websocket.send('{"eof" : 1}')
        print(await websocket.recv())
        
    #        return accept
    
asyncio.get_event_loop().run_until_complete(gpu_approach(original_file_path, original_file_name))

RuntimeError: This event loop is already running